In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [2]:
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()

x_train = x_train.reshape((x_train.shape[0], -1)).astype('float32') / 255.0
x_test  = x_test.reshape((x_test.shape[0], -1)).astype('float32') / 255.0

In [3]:
input_dim = 784
hidden_dim = 64
sparsity_level = 0.05
lambda_sparse = 0.1

In [4]:
inputs = layers.Input(shape=(input_dim,))
encoded = layers.Dense(hidden_dim, activation='relu')(inputs)
decoded = layers.Dense(input_dim, activation='sigmoid')(encoded)

autoencoder = keras.Model(inputs, decoded)
encoder = keras.Model(inputs, encoded)

In [5]:
def sparse_loss(y_true, y_pred):
    mse_loss = tf.reduce_mean(keras.losses.MeanSquaredError()(y_true, y_pred))

    hidden_layer_output = encoder(y_true)
    mean_activation = tf.reduce_mean(hidden_layer_output, axis=0)

    kl_divergence = tf.reduce_sum(
        sparsity_level * tf.math.log(sparsity_level / (mean_activation + 1e-10)) +
        (1 - sparsity_level) * tf.math.log((1 - sparsity_level) / (1 - mean_activation + 1e-10))
    )

    return mse_loss + lambda_sparse * kl_divergence